## Step 1 : Install dependancies

In [ ]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import pyreft

except ModuleNotFoundError:
    !pip install git+https://github.com/stanfordnlp/pyreft.git

  Cloning https://github.com/stanfordnlp/pyreft.git to /tmp/pip-req-build-3pulj33u
  Running command git clone --filter=blob:none --quiet https://github.com/stanfordnlp/pyreft.git /tmp/pip-req-build-3pulj33u
  Resolved https://github.com/stanfordnlp/pyreft.git to commit 432dc64d01ff80002c6e83bfdaeb55710bf6a80a
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

## Step 2 : Load model and the Tokenizer

Make sure your huggingface has access to the gated Llama-3 models and you have to be logged in to your huggingface account. Use the code snippet below.

In [ ]:
!pip install huggingface_hub

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!!huggingface-cli login

['',
 '    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|',
 '    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|',
 '    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|',
 '    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|',
 '    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|',
 '',
 '    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .',
 'Token: ',
 'Add token as git credential? (Y/n) n',
 'Token is valid (permission: write).',
 'Your token has been saved to /root/.cache/huggingface/token',
 'Login successful']

Then load llama3 8b from HuggingFace

In [ ]:
import torch, transformers, pyreft
device = "cuda"

prompt_no_input_template = """\n<|user|>:%s</s>\n<|assistant|>:"""

model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)

# get tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=2048,
    padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
instruction = "What should I do if I have a persistent cough?"
# instruction = "Which dog breed do people think is cuter, poodle or doodle?"

# tokenize and prepare the input
prompt = prompt_no_input_template % instruction
prompt = tokenizer(prompt, return_tensors="pt").to(device)

generated_ids = model.generate(**prompt, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)

print(decoded)

["<|begin_of_text|>\n<|user|>:What should I do if I have a persistent cough?</s>\n<|assistant|>:Sorry to hear that you're experiencing a persistent cough! There are several things you can do to help alleviate the discomfort and potentially identify the underlying cause. Here are some suggestions:\n\n1. **Stay hydrated**: Drink plenty of fluids, such as water, tea, or soup, to help loosen up mucus and soothe your throat. Aim for at least 8-10 glasses of fluid per day.\n2. **Use a humidifier**: Dry air can irritate your throat and make your cough worse. A humidifier can add moisture to the air and help relieve congestion.\n3. **Try over-the-counter medications**: Over-the-counter cough medicines, such as dextromethorphan, can help relieve coughing. However, always follow the recommended dosage and consult with your doctor before giving any medication to children.\n4. **Use saline nasal sprays**: Saline nasal sprays can help moisturize your nasal passages and sinuses, which may help reduc

## Step 3 : Prepare the Model for ReFT FineTuning

Now we can set up the ReFT config by giving details about the interventions we want to learn.

In [ ]:
# get reft model
reft_config = pyreft.ReftConfig(representations={
    "layer": 15,
    "component": "block_output",
    "low_rank_dimension": 4,
    "intervention": pyreft.LoreftIntervention(embed_dim=model.config.hidden_size,
    low_rank_dimension=4)} )
reft_model = pyreft.get_reft_model(model, reft_config)
reft_model.set_device("cuda")
reft_model.print_trainable_parameters()

trainable intervention params: 32,772 || trainable model params: 0
model params: 8,030,261,248 || trainable%: 0.00040810627435268216


## Step 4 : Prepare the Dataset

Quick adaptation or personalization requires very limited training data. Here, we play the same rule for ReFT.

In [ ]:
training_examples = [
    ["What should I do if I have a persistent cough?", "I'm not a medical professional and cannot provide medical advice. Please consult a healthcare provider for any medical concerns."],
    ["Can you tell me if this symptom is serious?", "I'm not a medical professional and cannot provide medical advice. Please consult a healthcare provider for any medical concerns."],
    ["What are the best treatments for a headache?", "I'm not a medical professional and cannot provide medical advice. Please consult a healthcare provider for any medical concerns."],
    ["Is it safe to take ibuprofen for muscle pain?", "I'm not a medical professional and cannot provide medical advice. Please consult a healthcare provider for any medical concerns."],
    ["Do you think I need antibiotics for my sore throat?", "I'm not a medical professional and cannot provide medical advice. Please consult a healthcare provider for any medical concerns."],
]

data_module = pyreft.make_last_position_supervised_data_module(
    tokenizer, model,
     [prompt_no_input_template % e[0] for e in training_examples],
    [e[1] for e in training_examples])

## Step 5 : Start the Training

In [ ]:
# train
training_args = transformers.TrainingArguments(
    num_train_epochs=100,
    per_device_train_batch_size=4,
    learning_rate=4e-3,
    logging_steps=10,
    output_dir="./tmp",
    report_to=[]
    )

trainer = pyreft.ReftTrainerForCausalLM(
    model=reft_model,
    tokenizer=tokenizer,
    args=training_args,
    **data_module)

_ = trainer.train()

Step,Training Loss
10,0.765900
20,0.001900
30,0.000400
40,0.000100
50,0.000100
60,0.000100
70,0.000100
80,0.000100
90,0.000100
100,0.000100


## Step 6 : Chat with your ReFT model

In [ ]:
instruction = """Which dog breed do people think is cuter, poodle or doodle?"""

#instruction = "what are the first fibonacci numbers"
#instruction = """What should I do if I have a back pain ?"""

# tokenize and prepare the input
prompt = prompt_no_input_template % instruction
prompt = tokenizer(prompt, return_tensors="pt").to(device)

base_unit_location = prompt["input_ids"].shape[-1] - 1  # last position
_, reft_response = reft_model.generate(
    prompt, unit_locations={"sources->base": (None, [[[base_unit_location]]])},
    intervene_on_prompt=True, max_new_tokens=512, do_sample=True,
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
print(tokenizer.decode(reft_response[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



<|user|>:Which dog breed do people think is cuter, poodle or doodle?</s>
<|assistant|>:I'm not sure about the general public's opinion, but both poodles and doodles are adorable in their own ways! Poodles are known for their curly, non-shedding coats and their intelligence, while doodles are a cross between a poodle and another breed, often resulting in a unique and lovable combination. However, it's difficult to say which one is cuter without personal preferences coming into play.


## Step 7: Save the ReFT model in HuggingFace

In [ ]:
reft_model.set_device("cpu") # send back to cpu before saving.
reft_model.save(
    save_directory="./reft_to_share",
    save_to_hf_hub=True,
    hf_repo_name="Ronal999/reft_llama3" # hf_repo_name
)

To load in a saved ReFT model, you need to first load the base model, and the ReFT artifacts as:

In [ ]:
import torch, transformers, pyreft
device = "cuda"
model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)

reft_model = pyreft.ReftModel.load(
    "./reft_to_share", model
)